In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

%matplotlib inline

In [5]:
df = pd.read_csv("AmesHousingWithAddress.csv")

df = df[["SalePrice", "Lot Area", "Gr Liv Area", "Total Bsmt SF", "Open Porch SF", "Wood Deck SF", "Garage Area", 
        "Bedroom AbvGr", "Full Bath", "Bsmt Full Bath", "Kitchen AbvGr", "Fireplaces", "Neighborhood", 
        "Year Built", "Year Remod/Add"]]

le = LabelEncoder()
df["Neighborhood"] = le.fit_transform(df["Neighborhood"])

df["Bsmt Full Bath"] = df["Bsmt Full Bath"].fillna(0)

df.head()

,SalePrice,Lot Area,Gr Liv Area,Total Bsmt SF,Open Porch SF,Wood Deck SF,Garage Area,Bedroom AbvGr,Full Bath,Bsmt Full Bath,Kitchen AbvGr,Fireplaces,Neighborhood,Year Built,Year Remod/Add
0,215000,31770,1656,1080,62,210,528,3,1,1.0,1,2,13,1960,1960
1,105000,11622,896,882,0,140,730,2,1,0.0,1,0,13,1961,1961
2,172000,14267,1329,1329,36,393,312,3,1,0.0,1,0,13,1958,1958
3,244000,11160,2110,2110,0,0,522,3,2,1.0,1,2,13,1968,1968
4,189900,13830,1629,928,34,212,482,3,2,0.0,1,1,8,1997,1998


In [51]:
def conformal_prediction(df, regressor, gamma, confidence):
    
    data_0 = df.sample(frac = 0.2, axis = "index")

    data_1 = df[~df.isin(data_0)].dropna(how = "any").sample(frac = 0.25, axis = "index")

    data_2 = df[~df.isin(data_0)].dropna(how = "any")
    data_2 = data_2[~data_2.isin(data_1)].dropna(how = "any").sample(frac = 0.333, axis = "index")

    data_3 = df[~df.isin(data_0)].dropna(how = "any")
    data_3 = data_3[~data_3.isin(data_1)].dropna(how = "any")
    data_3 = data_3[~data_3.isin(data_2)].dropna(how = "any").sample(frac = 0.5, axis = "index")

    data_4 = df[~df.isin(data_0)].dropna(how = "any")
    data_4 = data_4[~data_4.isin(data_1)].dropna(how = "any")
    data_4 = data_4[~data_4.isin(data_2)].dropna(how = "any")
    data_4 = data_4[~data_4.isin(data_3)].dropna(how = "any")

    cross_val_list = [data_0, data_1, data_2, data_3, data_4]

    for i in range(len(cross_val_list)):
        
        train_data = cross_val_list[0:3]
        train_data = pd.concat([train_data[0], train_data[1], train_data[2]])
        cal_data = cross_val_list[3]
        test_data = cross_val_list[4]       
        
        X_train = train_data[["Lot Area", "Gr Liv Area", "Total Bsmt SF", "Open Porch SF", "Wood Deck SF", "Garage Area", 
                            "Bedroom AbvGr", "Full Bath", "Bsmt Full Bath", "Kitchen AbvGr", "Fireplaces", "Neighborhood", 
                            "Year Built", "Year Remod/Add"]]
        y_train = train_data["SalePrice"]

        X_cal = cal_data[["Lot Area", "Gr Liv Area", "Total Bsmt SF", "Open Porch SF", "Wood Deck SF", "Garage Area", 
                            "Bedroom AbvGr", "Full Bath", "Bsmt Full Bath", "Kitchen AbvGr", "Fireplaces", "Neighborhood", 
                            "Year Built", "Year Remod/Add"]]
        y_cal = cal_data["SalePrice"]

        X_test = test_data[["Lot Area", "Gr Liv Area", "Total Bsmt SF", "Open Porch SF", "Wood Deck SF", "Garage Area", 
                            "Bedroom AbvGr", "Full Bath", "Bsmt Full Bath", "Kitchen AbvGr", "Fireplaces", "Neighborhood", 
                            "Year Built", "Year Remod/Add"]]
        y_test = test_data["SalePrice"]

        reg = regressor
        
        reg2 = RandomForestRegressor()

        reg.fit(X_train, y_train)
        train_pred = reg.predict(X_train)
        y_cal_pred = reg.predict(X_cal)
        y_test_pred = reg.predict(X_test)
        
        train_resid = abs((y_train - train_pred))
        train_mu = np.log(abs(y_train - train_pred + 0.05))
        #print(train_mu.to_string())
        reg2.fit(X_train, train_mu)
        
        mu_cal_pred = reg2.predict(X_cal)
        mu_test_pred = reg2.predict(X_test)

        # Nonconformity Measure
        ncm = abs((y_cal - y_cal_pred) / (gamma + np.exp(mu_cal_pred)))
        ncm = list(ncm)
        
        q = np.quantile(ncm, confidence)
        
        lower = y_test_pred - (q * (gamma + np.exp(mu_test_pred)))
        upper = y_test_pred + (q * (gamma + np.exp(mu_test_pred)))
        global pred_intervals
        pred_intervals = list(zip(lower, upper))

        covered_count = 0
        not_covered_count = 0

        y_test.reset_index(drop = True, inplace = True)

        for i in range(len(y_test)):
            if (y_test[i] >= pred_intervals[i][0]) & (y_test[i] <= pred_intervals[i][1]):
                covered_count += 1
            else:
                not_covered_count += 1

        global coverage
        coverage = covered_count / (covered_count + not_covered_count)

        global int_size
        int_size = pred_intervals[0][1] - pred_intervals[0][0]

        cross_val_list.append(cross_val_list.pop(0))

In [52]:
regressors = [LinearRegression(), GradientBoostingRegressor(), RandomForestRegressor()]

confidences = [0.9, 0.95, 0.99]
gammas = [0, 2500, 5000, 10000, 20000, 40000]

In [53]:
n_trials = 100

for reg in regressors:
    for conf in confidences:
        for gamma in gammas:
            total_cvg = 0
            int_size_list = []
            for trial in range(n_trials):
                conformal_prediction(df, reg, gamma, conf)
                total_cvg += coverage
                int_size_list.append(int_size)
                #total_int_size += int_size
            print("Model: " + str(reg))
            print("Confidence: " + str(conf*100) + "%")
            print("Gamma Value: " + str(gamma))
            print("Average Coverage: " + str(total_cvg / n_trials))
            print("Median Interval Width: " + str(np.median(int_size_list)))
            print("--------------------------------------------------------------------")
            #print("Average Interval Width: " + str(total_int_size / n_trials))

Model: LinearRegression()
Confidence: 90.0%
Gamma Value: 0
Average Coverage: 0.900694143167028
Median Interval Width: 78742.55642767373
--------------------------------------------------------------------
Model: LinearRegression()
Confidence: 90.0%
Gamma Value: 2500
Average Coverage: 0.8989804772234276
Median Interval Width: 67437.69467277353
--------------------------------------------------------------------
Model: LinearRegression()
Confidence: 90.0%
Gamma Value: 5000
Average Coverage: 0.8972234273318871
Median Interval Width: 69905.97234475761
--------------------------------------------------------------------
Model: LinearRegression()
Confidence: 90.0%
Gamma Value: 10000
Average Coverage: 0.8977440347071587
Median Interval Width: 69948.75583363482
--------------------------------------------------------------------
Model: LinearRegression()
Confidence: 90.0%
Gamma Value: 20000
Average Coverage: 0.8990455531453364
Median Interval Width: 73709.75768376933
--------------------------

Model: RandomForestRegressor()
Confidence: 90.0%
Gamma Value: 10000
Average Coverage: 0.8993926247288502
Median Interval Width: 64741.68220261436
--------------------------------------------------------------------
Model: RandomForestRegressor()
Confidence: 90.0%
Gamma Value: 20000
Average Coverage: 0.8987201735357918
Median Interval Width: 67635.50706454308
--------------------------------------------------------------------
Model: RandomForestRegressor()
Confidence: 90.0%
Gamma Value: 40000
Average Coverage: 0.9012147505422992
Median Interval Width: 71069.70069778946
--------------------------------------------------------------------
Model: RandomForestRegressor()
Confidence: 95.0%
Gamma Value: 0
Average Coverage: 0.9447071583514095
Median Interval Width: 95632.05913682313
--------------------------------------------------------------------
Model: RandomForestRegressor()
Confidence: 95.0%
Gamma Value: 2500
Average Coverage: 0.9472234273318864
Median Interval Width: 75607.16148602325